## Global variables

player_list = []
button #deal button
shuffled_deck
last_move
dealt_cards
stack
pot
bet_storage
last_action
in_play 

## Create players

In [19]:
# Creates a list of players, with the length matching the number of players in the game

def create_players(playernum):
    players = [player for player in range(playernum)]
    
    return players

## Display board

In [1]:
"Hello"

'Hello'

## Deal button

In [22]:
# Sets a position of the dealer button

def deal_button(players, player_index):
    
    players[player_index] = 'D'
    
    return players

## Create shuffle deck

In [23]:
import random
# Creates a shuffled deck fo 52 cards from which we will pop when we deal out cards
def create_shuffle_deck():
    # Map card to number
    cards = ['A', '2', '3', '4', '5', '6', '7', '8', '9', '10', 'J', 'Q', 'K']
    suits = ["\u2663", "\u2665", "\u2666", "\u2660"]
    
    deck = [(card + suit) for card in cards for suit in suits]
    # Shuffle deck    
    
    random.shuffle(deck)
#     print(deck)
    
    return deck

## Stack for each player

In [24]:
# Sets the starting stack for each player
def starting_stack(players):
    starting_stack = int(input('What should the starting stack be? '))
    stacks = [starting_stack for i in players]
    return stacks

## Deal cards

In [25]:
# Creates a list of tuples where each tuple contains two cards for each player, being their hand
def deal_cards(deck, players):
    enum = enumerate(players)
    lst = []
    for player in enum:
        card1 = deck.pop()
        card2 = deck.pop()
        lst.append((card1, card2))
        
    return lst

## Bet

In [26]:
# This function is run when a player bets
# It updates the stack of the player, checks so the bet amount is valid and updates the bet_storage list
def bet(bet_storage_list, stacks_list, player_index):
    
    bet = int(input('What do you want to bet? '))
    while bet > stacks_list[player_index]:
        print("You cant bet more than you have in your stack")
        bet = int(input('What do you want to bet? '))
    else:
        stacks_list[player_index] -= bet
        bet_storage_list[player_index] += bet
        
    return bet_storage_list, stacks_list

## Bet blind

In [27]:
# This automatically puts in the blinds, updates the stacks of each player as well as bet storage and last_action
def bet_blind (button, stacks, small_blind, big_blind, bet_storage, last_action):
    for index, stack in enumerate(stacks):
        if button.index('D') + 1 == index:
            stack -= small_blind
            stacks[index] = stack
            bet_storage[index] = small_blind
        elif button.index('D') + 2 == index:
            stack -= int(big_blind)
            stacks[index] = stack
            bet_storage[index] = big_blind
            last_action[index] = 1
            
    return (stacks, bet_storage, last_action)

## Whose turn

In [28]:
def whose_turn(last_action_list):   #Return a list where the turn has moved one to the right, need to assign to variable
    copy = last_action_list.copy()
    copy[(last_action_list.index(1) + 1) % (len(last_action_list))] = 1
    copy[last_action_list.index(1)] = 0
    return copy

## Display hand

In [417]:
# Lets the player choose if they want to see their hand or not, applicable depending on FE
def display_hand(player_cards_list, player_index):
    choice = input('"y" for display hand, "n" to skip').lower()
    if choice == 'y':
        return player_cards_list[player_index]

## Check

In [30]:
# Should run when a player checks, updates check_list
def check(check_list, player_index):
    check_list[player_index] = 1
    return check_list

## Choose move

In [495]:
# Run when it is a certain players turn, lets the player choose which move to make depending on situation
# Then calls the function for the given move

def choose_move(bet_storage_list, stacks_list, in_play_list, check_list, player_index, last_action_type_list):
    if bet_storage_list.count(0) == len(bet_storage_list):
        move = input('What do you want to do? [Check] [Bet]').lower()
        
        if move == 'check':
            print(f"Player {player_index} checked")
            check(check_list, player_index)
            last_action_type_list[player_index] = 'check'
        elif move == 'bet':
            bet(bet_storage_list, stacks_list, player_index)
            last_action_type_list[player_index] = 'bet'
    else:
        move = input('What do you want to do? [Fold] [Call] [Raise]').lower()
        
        if move == 'fold':
            fold_move(in_play_list, player_index)
            last_action_type_list[player_index] = 'fold'
        elif move == 'call':
            call_move(bet_storage_list, stacks_list, player_index)
            last_action_type_list[player_index] = 'call'
        elif move == 'raise':
            raise_move(bet_storage_list, stacks_list, player_index)
            last_action_type_list[player_index] = 'raise'
            
    

## Fold

In [32]:
#Runs when a player folds, sets in_play for the players index to 0 and returns in_play_list

def fold_move(in_play_list, player_index):
    in_play_list[player_index] = 0
    return in_play_list

## Raise_move

In [33]:
# Calculates what the minimum allowed raise size is 
# Checks if the bet is over stack size 
# Updates stack and bet storage

def raise_move(bet_storage_list, stacks_list, player_index):
    
    # bet storage copy
    storage_copy = bet_storage_list.copy()
    # Bet enough
    first = max(storage_copy)
    storage_copy.pop(storage_copy.index(first))
    second = max(storage_copy)
    
    min_raise = (first - second) + first
    # print(first, second, min_raise)

    rais = int(input('What do you want to raise to? '))
    
    while rais > stacks_list[player_index] or rais < min_raise:
        if rais > stacks_list[player_index]:
            print(f"You cant raise more than you have in your stack {stacks_list[player_index]}")
            rais = int(input('What do you want to raise? '))
        elif rais < min_raise:
            print(f"The minimum raise is: {min_raise}")
            rais = int(input('What do you want to raise? '))
    else:
        stacks_list[player_index] -= rais
    
    bet_storage_list[player_index] += rais
        
    return bet_storage_list, stacks_list

## Call

In [34]:
# Updates bet storage and stacks and determines the amount to call

def call_move(bet_storage_list, stacks_list, player_index):
    largest_bet = max(bet_storage_list)
    to_call = largest_bet - bet_storage_list[player_index]
    stacks_list[player_index] -= to_call
    bet_storage_list[player_index] +=to_call
    
    
    return bet_storage_list, stacks_list

## Flop, turn and river

In [35]:
# Generates flop
def flop(deck):
    return[deck.pop() for i in range(3)]

In [36]:
# generates turn
def turn(deck):
    return [deck.pop()]

In [37]:
# generates river
def river(deck):
    return [deck.pop()]

## update_last_action

In [92]:
# Sets last_action on the dealer button, so the next round has to begin with moving last action to the one who has action
def update_last_action(dealer_list, last_action_list):
    last_action_list = [0 for i in range(len(last_action_list))]
    last_action_list[dealer_list.index('D')] = 1
    return last_action_list
    

In [98]:
print(dealer_list)
print(last_action_list)

['D', 1, 2, 3]
[0, 0, 1, 0]


In [100]:
last_action_list = update_last_action(dealer_list, last_action_list)
last_action_list

[1, 0, 0, 0]

## Showdown

In [421]:
def display_player_cards(player_cards_list, player_index):
	cards = player_cards_list[player_index]

	cards = [
    	[' ┌─────────┐  ┌─────────┐'],
    	[' │ '+ cards[0][0]+'   	│  │ '+cards[1][0]+'   	│'],
    	[' │     	│  │     	│'],
    	[' │     	│  │     	│'],
    	[' │	'+cards[0][1]+'	│  │	'+cards[0][1]+'	│'],
    	[' │     	│  │     	│'],
    	[' │     	│  │     	│'],
    	[' │   	'+cards[0][0]+' │  │   	'+cards[1][0]+' │'],
    	[' └─────────┘  └─────────┘']
	]
	return cards


In [ ]:
def showdown(stacks_list, in_play_list, last_action_list, last_action_type_list, player_cards_list, check_list, player_index, dealer_list, board, pot):
    win_list = [0 for i in in_play_list] ##Creating a list where all the winner of the pot are set to one
    ##Checking last raiser, then display their hand and set their hand strength
    if 'raise' in last_action_type_list:
        display_player_cards(player_cards_list, last_action_type_list.index('raise'))
        poker_hand = hand_strength(player_cards_list[last_action_type_list.index('raise')], board)
        print(f"Player {last_action_type_list.index('raise')} has a {poker_hand}")
        poker_hand_strength = hand_strength_list.index(poker_hand)
        
        ##For the players after the first raiser, until the end of the list, comparing their handstrength
        ##If the hand is better or equal, display their hand and update poker hand and poker hand strength
        for index, player in enumerate(in_play_list[last_action_type_list.index('raise') + 1:]):
            if player == 1 and hand_strength(player_cards_list[index], board) <= poker_hand_strength:
                display_player_cards(player_cards_list, index)
                poker_hand = hand_strength(player_cards_list[index], board)
                print(f"Player {index} has a {poker_hand}")
                poker_hand_strength = hand_strength_list.index(poker_hand)
        
        #Doing the same operation but from the beginning of the list until the last raiser
        for index, player in enumerate(in_play_list[:last_action_type_list.index('raise')]):
            if player == 1 and hand_strength(player_cards_list[index], board) <= poker_hand_strength:
                display_player_cards(player_cards_list, index)
                poker_hand = hand_strength(player_cards_list[index], board)
                print(f"Player {index} has a {poker_hand}")
                poker_hand_strength = hand_strength_list.index(poker_hand)
        
        
    # if their was no raising the last round, performing the same operations but for the last bettor           
    elif 'bet' in last_actiojn_type_list:
        if 'bet' in last_action_type_list:
            display_player_cards(player_cards_list, last_action_type_list.index('bet'))
            poker_hand = hand_strength(player_cards_list[last_action_type_list.index('bet')], board)
            print(f"Player {last_action_type_list.index('bet')} has a {poker_hand}")
            poker_hand_strength = hand_strength_list.index(poker_hand)
        
        ##For the players after the first raiser, until the end of the list, comparing their handstrength
        ##If the hand is better or equal, display their hand and update poker hand and poker hand strength
        for index, player in enumerate(in_play_list[last_action_type_list.index('bet') + 1:]):
            if player == 1 and hand_strength(player_cards_list[index], board) <= poker_hand_strength:
                display_player_cards(player_cards_list, index)
                poker_hand = hand_strength(player_cards_list[index], board)
                print(f"Player {index} has a {poker_hand}")
                poker_hand_strength = hand_strength_list.index(poker_hand)
        
        #Doing the same operation but from the beginning of the list until the last raiser
        for index, player in enumerate(in_play_list[:last_action_type_list.index('bet')]):
            if player == 1 and hand_strength(player_cards_list[index], board) <= poker_hand_strength:
                display_player_cards(player_cards_list, index)
                poker_hand = hand_strength(player_cards_list[index], board)
                print(f"Player {index} has a {poker_hand}")
                poker_hand_strength = hand_strength_list.index(poker_hand)
    
    #Only executing the esle statement if noone raised or bet, which means the betting storage will be empty
    else:
        best_hand = 10
        for index, player in enumerate(in_play_list[dealer_list.index('D')]):
             if player:
                    if hand_strength(player_cards_list[index], board) <= best_hand:
                        if hand_strength(player_cards_list[index], board) < best_hand: #Checking if the hand is the only best
                            win_list = [0 for i in win_list]
                            win_list[index] = player
                        best_hand = hand_strength(player_cards_list[index], board)
                        display_player_cards(player_cards_list, index)
                        win_list[index] = player
                       
                        
    
    #Giving the pot to the winner(s) through checking if their value in win_list is 1
    for index, player in enumerate(win_list):
        amount_won = pot / (win_list.count(1))
        if player:
            stacks_list[index] += amount_won
    
                        

## Hand strength

In [130]:
# THis is an idea of cleaner code but have not had time to make it work
def hand_strength_clean(player_cards, board):
    hand_strength_list = ['straight_flush', 'quads', 'full_house', 'flush', 'straight', 'trips', 'two_pair', 'pair']
    for hand in hand_strength_list:
        if eval(hand_strength_list[hand])(player_cards, board):
            return eval(hand_strength_list[hand])(player_cards_list, board)
            break
            
hand_strength_list = ['straight_flush', 'quads', 'full_house', 'flush', 'straight', 'trips', 'two_pair', 'pair']

## card_conversion

In [498]:
## THis is a functino for converting the value of a card to an integer, J = 11, Q=12, K=13 and A=14
## To be able to have a striaght A-5 we need to more logic and an exception, which will be an add on


def card_conversion(card):
    if card[0] == 'A':
        return 14
    elif card[0] == 'K':
        return 13
    elif card[0] == 'Q':
        return 12
    elif card[0] == 'J':
        return 11
    elif card[0] == '1':
        return 10
    else:
        return int(card[0])

## Hand strength

In [ ]:
##Hand Strength, the cell also contains all the functions for each type of hand

In [398]:
def straight_sort(hand):
    hand.sort(key = lambda x: x.split()[0])

In [520]:
import itertools

def straight_flush(player_cards, board):
    cards = list(player_cards) + board
    hands = list(itertools.combinations(cards, 5))
    for hand in hands:
        for card in hand:
            if [card[-1] for card in hand].count(card[-1]) == 5\
            and len(set([card_conversion(card) for card in hand])) == 5\
            and max([card_conversion(card) for card in hand]) - min([card_conversion(card) for card in hand]):
                return 'straight_flush'


def quads(player_cards, board):
    cards = list(player_cards) + board
    hands = list(itertools.combinations(cards, 5))
    for hand in hands:
        for card in hand:
            if [card[0] for card in hand].count(card[0]) == 4:
                return 'quads'


def flush(player_cards, board):
    cards = list(player_cards) + board
    hands = list(itertools.combinations(cards, 5))
    for hand in hands:
        for card in hand:
            if [card[-1] for card in hand].count(card[-1]) == 5:
                return 'flush'


def full_house(player_cards, board):
    cards = list(player_cards) + board
    hands = list(itertools.combinations(cards, 7))
    for hand in hands:
        for card in hand:
            if [card[0] for card in hand].count(card[0]) == 3:
                a = card[0]
                if len({card[0] for card in hand if card[0] != a}) < 4:
                    return 'full_house'


def straight(player_cards, board):
    cards = list(player_cards) + board
    hands = list(itertools.combinations(cards, 5))
    for hand in hands:
        if len(set([card_conversion(card) for card in hand])) == 5\
        and max([card_conversion(card) for card in hand]) - min([card_conversion(card) for card in hand]) == 4:
            return 'straight'


def trips_or_two_pair(player_cards, board):
    cards = list(player_cards) + board
    hands = list(itertools.combinations(cards, 5))
    print(hands[0])
    for hand in hands:
        if len(set([(card_conversion(card)) for card in hand])) == 3:
            longest_set = []
            for card in [(card_conversion(card)) for card in hand]:
                if [(card_conversion(card)) for card in hand].count(card) == 3:
                    return 'trips'
                elif [(card_conversion(card)) for card in hand].count(card) == 2:
                    return 'two pair'


                
def pair(player_cards, board):
    cards = list(player_cards) + board
    hands = list(itertools.combinations(cards, 5))
    for hand in hands:
        for card in hand:
            if [card[0] for card in hand].count(card[0]) == 2:
                return 'one pair'
    return False

            
    
def hand_strength(player_cards, board):
    hand_strength_list = ['straight_flush', 'quads', 'full_house', 'flush', 'straight', 'trips','two pair', 'pair']
    for hand in hand_strength_list:
        if hand == 'straight_flush':
            if straight_flush(player_cards, board):
                return 'quads'
        elif hand == 'quads':
            if quads(player_cards, board):
                return 'quads'
        elif hand == 'full_house':
            if full_house(player_cards, board):
                return 'full_house'
        elif hand == 'flush':
            if flush(player_cards, board):
                return 'flush'
        elif hand == 'straight':
            if straight(player_cards, board):
                return 'straight'
        elif hand == 'trips':
            if trips_or_two_pair(player_cards, board) == 'trips':
                return 'trips'
            elif trips_or_two_pair(player_cards, board) == 'two pair':
                return 'two pair'
            
        elif hand == 'pair':
            if pair(player_cards, board):
                return 'pair'
    
        

In [308]:
board_test = ['9♦', '9♦', '9♥','9♠','A♠']

In [406]:
straight_flush_test = ['9♦', '5♦', '7♦','6♠','A♠']

In [472]:
quads_test = ['9♦', '9♦', '9♥','9♠','A♠']

In [488]:
boat_test = ['9♦', '9♦', '9♥','A♠','A♠']

In [384]:
trips_twopair_test = ['9♦', '9♦', 'J♥','J♠','A♠']

In [404]:
hand_test = ['7♠', '8♦']

In [407]:
def straight_flush(player_cards, board):
    cards = player_cards + board
    hands = list(itertools.combinations(cards, 5))
    for hand in hands:
        for card in hand:
            if [card[-1] for card in hand].count(card[-1]) == 5\
            and len(set([card_conversion(card) for card in hand])) == 5\
            and max([card_conversion(card) for card in hand]) - min([card_conversion(card) for card in hand]):
                return 'straight_flush'
    return False

In [477]:
def quads(player_cards, board):
    cards = player_cards + board
    hands = list(itertools.combinations(cards, 5))
    print("hej quads")
    for hand in hands:
        for card in hand:
            if [card[0] for card in hand].count(card[0]) == 4:
                return 'quads'

In [489]:
hand_strength(hand_test, boat_test)

'full_house'

In [354]:
straight(hand_test, straight_flush_test)

'straight'

## Betting round

In [496]:
# Runs when there is a betting round, so once before flop, once after flop, once after turn and finally once after river
# One iteration is for one player
def betting_round(bet_storage_list, stacks_list, in_play_list, last_action_list, player_cards_list, check_list, player_index):
    
    while (set(bet_storage_list) == {0} or len({bet_storage_list[i] for i in [index for index, item in enumerate(in_play_list) if item == 1]}) > 1) \
    and in_play_list.count(1) != [check_list[i] for i in [index for index, item in enumerate(in_play_list) if item == 1]].count(1):
        
        ## Update last_player_list to current player
        last_action_list = whose_turn(last_action_list)
      

        ## Display card
        player_index = last_action_list.index(1)
        ## Checking if the player has not folded yet
        if in_play_list[player_index] == 0:
            continue
        a = display_hand(player_cards_list, player_index)
        print(f'Player {player_index} cards: {a}')

        ## Choose move
        choose_move(bet_storage_list, stacks_list, in_play_list, check_list, player_index, last_action_type_list)




## Main

In [ ]:
print('Welcome to Texas hold em')


# players = int(input('How many players? '))
players = 4

## Return players
player_list = create_players(players)

## Create last action list, to use later
## Creating other necessary lists
last_action_list = [0 for x in player_list]
bet_storage_list = [0 for x in player_list]
in_play_list = [1 for x in player_list]
check_list = [0 for x in player_list]
pot = 0
hand_strength_list = ['straight_flush', 'quads', 'full_house', 'flush', 'straight', 'trips', 'pair']
last_action_type_list = ['' for x in player_list]

## Create stacks
stacks_list = starting_stack(player_list)
stacks_list

while len([i for i in stacks_list if i > 0]):

## Assigns the dealer button to player of index 0
    dealer_list = deal_button(player_list, 0)

## Creating shuffled deck
    deck = create_shuffle_deck()



## Ask for bet blind amount
# big_blind = int(input('What\'s the big blind? '))
    big_blind = 40
    small_blind = big_blind / 2


## Deal cards
    player_cards_list = deal_cards(deck, player_list)


    ## Bet blind
    bet_blind(dealer_list, stacks_list, small_blind, big_blind, bet_storage_list, last_action_list)

# The while loop checks if all the bets for the player that are in play are  not equal, or if all player in play have not checked
    betting_round(bet_storage_list, stacks_list, in_play_list, last_action_list, player_cards_list, check_list, player_index)
    last_action_list = update_last_action(dealer_list, last_action_list)
    
## Update pot
    pot += sum(bet_storage_list)

## Reset bet_storage_list
    bet_storage_list = [0 for x in player_list]

##Check if there is more than one player left in the pot
    if in_play_list.count(1) < 2:
        stacks_list[in_play_list.index(1)] += pot
        continue
    
    
    ## Display flop
    board = flop(deck)
    print(f'Flop: {board}')
    print(stacks_list)
    print(pot)

## New betting round
    betting_round(bet_storage_list, stacks_list, in_play_list, last_action_list, player_cards_list, check_list, player_index)
    last_action_list = update_last_action(dealer_list, last_action_list)

## Update pot
    pot += sum(bet_storage_list)

##Check if there is more than one player left in the pot
    if in_play_list.count(1) < 2:
        stacks_list[in_play_list.index(1)] += pot
        continue

    ## Display turn
    board += turn(deck)
    print(f'Turn: {board}')

## New betting round
    betting_round(bet_storage_list, stacks_list, in_play_list, last_action_list, player_cards_list, check_list, player_index)
    last_action_list = update_last_action(dealer_list, last_action_list)

## Update pot
    pot += sum(bet_storage_list)

## Reset bet_storage_list
    bet_storage_list = [0 for x in player_list]

##Check if there is more than one player left in the pot
    if in_play_list.count(1) < 2:
        stacks_list[in_play_list.index(1)] += pot
        continue

    ## Display river
    board += river(deck)
    print(f'River: {board}')

    ## New betting round
    betting_round(bet_storage_list, stacks_list, in_play_list, last_action_list, player_cards_list, check_list, player_index)
    last_action_list = update_last_action(dealer_list, last_action_list)

    ## Update pot
    pot += sum(bet_storage_list)

    ## Reset bet_storage_list
    bet_storage_list = [0 for x in player_list]

    ## Showdown (The one being called shows first, then in order. The winner adds the pot to their stack)
    showdown(stacks_list, in_play_list, last_action_list, last_action_type_list, player_cards_list, player_index, dealer_list, board, pot)



Welcome to Texas hold em
What should the starting stack be? 1000
"y" for display hand, "n" to skipy
Player 3 cards: ('J♣', '6♠')
What do you want to do? [Fold] [Call] [Raise]fold
"y" for display hand, "n" to skipfold
Player 0 cards: None
What do you want to do? [Fold] [Call] [Raise]fold
"y" for display hand, "n" to skipy
Player 1 cards: ('K♥', 'J♥')
What do you want to do? [Fold] [Call] [Raise]fold


In [ ]:
print(stacks_list)

In [516]:
dealer_list.index('D')

0

In [517]:
in_play_list[dealer_list.index('D')]

0